# Experiment 2: Acquiring time-domain IQ data 



In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
%matplotlib inline

Project path set to: G:\Shared drives\Quasiparticles_2\Radiator\032525\L1A_clearingtone_ON_7GHz_7db__nonlinear_radiator_6K_MXC_26mk_part2


In [18]:
from flux_fit import *
from utils import *

In [19]:
!python -m srsinst.dc205

## Setting up the Experiment

Setup the project and experiment names

In [ ]:
project_name = "Andreev_Spectroscopy"
device_name = "L1A"
run_id = 0
T_MXC = 31
T_Rad = None
experiment_name = f"{device_name}_trapping_measurement_RUN{run_id}"

drive_path = r"E:\Quasiparticles"
data_path = set_project(r"{}\{}".format(drive_path, project_name), f"{experiment_name}")

# Constants
SPATH = data_path
FIG_PATH = os.path.join(SPATH, "Figures")
LOG_FILE = os.path.join(SPATH, f'MEASUREMENTLOG_{time.strftime("%Y%m%d_%H%M%S")}.log')
PATH_TO_EXE = r'C:/Users/LFL/lflPython/AlazarDrivers/CS_Average/x64/Release/ATS9371_CS_Average.exe'

# Ensure directories exist
os.makedirs(FIG_PATH, exist_ok=True)

# Setup logging
logging.basicConfig(filename=LOG_FILE, filemode='w', level=logging.INFO)

### Instrument Setup

Connect to the Labber server and initialize the instruments needed for the experiment

In [ ]:
client = Labber.connectToServer(timeout=None)
instrument_list = client.getListOfInstruments()
instruments = create_instrument_connections(client, instrument_list)

In [ ]:
vs = connect_SRS()
VNA = instruments['Agilent Network Analyzer E5071B']
DA = instruments['Vaunix Lab Brick Digital Attenuator']
SMU = instruments['Keithley 2400 SourceMeter']
LO= instruments['SignalCore SC5511A Signal Generator_10002F25']
Drive= instruments['SignalCore SC5511A Signal Generator_1000334C']
TWPA_PUMP = instruments['Rohde&Schwarz RF Source']

initialize_instruments(VNA, DA, SMU, LO, Drive, vs, TWPA_PUMP)

Make sure that the instruments are set to the correct state.

- The VNA is turned off
- The TWPA pump is set to the correct frequency and power
- The LO is set to the correct frequency and power
- The SRS is set to the correct voltage
- The Alazar is configured to acquire the correct number of traces, with the correct acquisition length, sample rate, and average time cycle.

In [ ]:
turn_off_vna()

In [ ]:
set_TWPA_pump(f=6.04, power=27)

In [ ]:
num_traces=1
acquisitionLength_sec=5
origRateMHz=300
sampleRateMHz=10
averageTimeCycle=0
lowerBound=14 # dB for DA attenuator
upperBound=30# dB for DA attenuator

span = 10e6 # Hz
electrical_delay = 82.584e-9 # s
vna_power = 5
vna_avg = 20
f_d_power = 16


In [21]:
f_clearing, P_clearing = None, None

Setting the LO and turning off the VNA

In [ ]:
set_LO_tone(f_drive, f_d_power)
turn_off_vna()

In [ ]:
logging.info(f'\nStarting Alazar acquisition at phi = {phi:.3f} while driving at {f_drive:.6f} GHz')
savefile = acquire_IQ_data(phi, f_clearing, P_clearing, num_traces, acquisitionLength_sec, origRateMHz, sampleRateMHz, averageTimeCycle, lowerBound, upperBound)
write_metadata(savefile, acquisitionLength_sec, origRateMHz, f_drive, voltage, T_MXC, T_Rad, phi, f_clearing, P_clearing)